In [19]:
import os
import glob
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Paths
ROOT_DATA_DIR = r"C:\Users\LENOVO\OneDrive\Desktop\Project"
TEST_DIR = os.path.join(ROOT_DATA_DIR, "CLASSIFICATION_ROIS_128x128")
MODEL_PATH = os.path.join(ROOT_DATA_DIR, "best_efficientnet_b4.pth")
SAVE_DIR = os.path.join(ROOT_DATA_DIR, "ANNOTATED_TEST_RESULTS_VISUALIZED")
os.makedirs(SAVE_DIR, exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", DEVICE)


Using: cpu


In [20]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [21]:
class PCBTestDataset(Dataset):
    def __init__(self, dir_path, transform=None):
        self.transform = transform
        self.img_paths = glob.glob(os.path.join(dir_path, "*", "*.png"))
        self.labels = [os.path.basename(os.path.dirname(p)) for p in self.img_paths]
        
        self.classes = sorted(list(set(self.labels)))
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        path = self.img_paths[idx]
        img = Image.open(path).convert("RGB")
        label_name = os.path.basename(os.path.dirname(path))
        label_idx = self.class_to_idx[label_name]

        if self.transform:
            img = self.transform(img)

        return img, label_idx, label_name, path


In [14]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [15]:
class PCBTestDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.transform = transform

        # Load images inside subfolders (each folder = class)
        self.img_paths = glob.glob(os.path.join(directory, "*", "*.png"))
        self.labels = [os.path.basename(os.path.dirname(p)) for p in self.img_paths]

        # Class names sorted (consistent with training)
        self.classes = sorted(list(set(self.labels)))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        path = self.img_paths[idx]
        img = Image.open(path).convert("RGB")

        label_name = os.path.basename(os.path.dirname(path))
        label = self.class_to_idx[label_name]

        if self.transform:
            img = self.transform(img)

        return img, label, label_name, path


In [22]:
test_dataset = PCBTestDataset(TEST_DIR, transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Detected Classes:", test_dataset.classes)


Detected Classes: ['Missing_hole', 'Mouse_bite', 'Open_circuit', 'Short', 'Spur', 'Spurious_copper']


In [23]:
model = timm.create_model(
    "efficientnet_b4",
    pretrained=False,
    num_classes=len(test_dataset.classes)
)

model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

print(" Model Loaded Successfully!")


 Model Loaded Successfully!


In [18]:
MODEL_PATH = os.path.join(ROOT_DATA_DIR, "best_efficientnet_b4.pth")

model = EfficientNet.from_name('efficientnet-b4', num_classes=len(test_dataset.classes))
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

print("Model Loaded Successfully!")


RuntimeError: Error(s) in loading state_dict for EfficientNet:
	Missing key(s) in state_dict: "_conv_stem.weight", "_bn0.weight", "_bn0.bias", "_bn0.running_mean", "_bn0.running_var", "_blocks.0._depthwise_conv.weight", "_blocks.0._bn1.weight", "_blocks.0._bn1.bias", "_blocks.0._bn1.running_mean", "_blocks.0._bn1.running_var", "_blocks.0._se_reduce.weight", "_blocks.0._se_reduce.bias", "_blocks.0._se_expand.weight", "_blocks.0._se_expand.bias", "_blocks.0._project_conv.weight", "_blocks.0._bn2.weight", "_blocks.0._bn2.bias", "_blocks.0._bn2.running_mean", "_blocks.0._bn2.running_var", "_blocks.1._depthwise_conv.weight", "_blocks.1._bn1.weight", "_blocks.1._bn1.bias", "_blocks.1._bn1.running_mean", "_blocks.1._bn1.running_var", "_blocks.1._se_reduce.weight", "_blocks.1._se_reduce.bias", "_blocks.1._se_expand.weight", "_blocks.1._se_expand.bias", "_blocks.1._project_conv.weight", "_blocks.1._bn2.weight", "_blocks.1._bn2.bias", "_blocks.1._bn2.running_mean", "_blocks.1._bn2.running_var", 

In [25]:
model = timm.create_model(
    "efficientnet_b4",
    pretrained=False,
    num_classes=len(test_dataset.classes)
)

model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

print(" Model Loaded Successfully!")


 Model Loaded Successfully!


In [28]:
def unnormalize(img_tensor):
    img = img_tensor.cpu().numpy().transpose(1, 2, 0)
    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    return np.clip(img, 0, 1)


In [27]:
true_labels = []
pred_labels = []

plt.switch_backend('Agg')

for imgs, labels, label_names, paths in test_loader:
    imgs = imgs.to(DEVICE)

    with torch.no_grad():
        outputs = model(imgs)
        probs = F.softmax(outputs, dim=1)
        preds = torch.argmax(probs, 1)

    preds_np = preds.cpu().numpy()
    labels_np = labels.numpy()

    true_labels.extend(labels_np)
    pred_labels.extend(preds_np)

    # save annotated predictions
    for i in range(len(imgs)):
        img_np = unnormalize(imgs[i])
        true_name = test_dataset.classes[labels_np[i]]
        pred_name = test_dataset.classes[preds_np[i]]
        confidence = float(torch.max(probs[i]).cpu()) * 100

        plt.figure(figsize=(3, 3))
        plt.imshow(img_np)
        plt.title(f"True: {true_name}\nPred: {pred_name} ({confidence:.1f}%)")
        plt.axis("off")

        filename = os.path.basename(paths[i]).replace(".png", "")
        save_path = os.path.join(SAVE_DIR, f"{filename}_T_{true_name}_P_{pred_name}.png")

        plt.savefig(save_path, bbox_inches="tight")
        plt.close()

print("Annotated predictions saved in:", SAVE_DIR)

Annotated predictions saved in: C:\Users\LENOVO\OneDrive\Desktop\Project\ANNOTATED_TEST_RESULTS_VISUALIZED


In [ ]:
print("\nFINAL EVALUATION")

acc = accuracy_score(true_labels, pred_labels)
print("Accuracy:", acc)

print("\nClassification Report:")
print(classification_report(true_labels, pred_labels, target_names=test_dataset.classes))

print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, pred_labels))

match_rate = (np.array(true_labels) == np.array(pred_labels)).mean() * 100
print(f"\nPrediction Match Rate: {match_rate:.2f}%")
